In [1]:
import torch
from torch import nn
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(2,4)
net(X)

tensor([[-0.2655],
        [-0.1365]], grad_fn=<AddmmBackward0>)

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0235,  0.2529,  0.1823, -0.3043,  0.3257, -0.3518, -0.0798,  0.1642]])), ('bias', tensor([-0.1809]))])


In [3]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1809], requires_grad=True)
tensor([-0.1809])


In [6]:
net[2].bias.grad==None

True

In [7]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [8]:
print(*[(name,param.shape) for name,param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].data

tensor([-0.1809])

In [10]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net
rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [11]:
rgnet(X)

tensor([[-0.3233],
        [-0.3233]], grad_fn=<AddmmBackward0>)

In [12]:
rgnet[0][3][2].bias.data

tensor([ 0.3430,  0.3294, -0.3193,  0.1736])

### 参数初始化

In [13]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0023, -0.0186,  0.0098,  0.0170]), tensor(0.))

In [14]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [16]:
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.5539,  0.3072, -0.4052, -0.5588],
        [-0.5232, -0.4168,  0.2407,  0.0285],
        [-0.5826, -0.0156, -0.0782,  0.1341],
        [-0.2168, -0.6695,  0.1157,  0.6154],
        [-0.5950, -0.6112, -0.0984,  0.3860],
        [-0.1224, -0.3026,  0.4753, -0.2462],
        [-0.6110, -0.3599, -0.5745,  0.3925],
        [ 0.3299, -0.6089, -0.2998,  0.4006]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [18]:
def my_init(m):
    if type(m)==nn.Linear:
        print('Init',*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data*=m.weight.data.abs()>=5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000,  8.6004,  0.0000],
        [-8.1047,  0.0000, -9.6280,  0.0000]], grad_fn=<SliceBackward0>)

In [19]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  9.6004,  1.0000])

### 参数绑定

In [20]:
shared=nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                    shared,nn.ReLU(),
                    shared,nn.ReLU(),
                    nn.Linear(8,1))
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=1000
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
